# Imports

In [1]:
import pandas as pd
from think_reason_learn.policy_induction import PolicyInduction, WeightTrainerConfig
from think_reason_learn.core.llms import GoogleChoice, OpenAIChoice

# Log level

In [2]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,  # You might want debug or info
    stream=sys.stdout,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    force=True,
)

logging.getLogger("google_genai.models").setLevel(logging.ERROR)
logging.getLogger("google_genai.models").propagate = False

# Data

In [3]:
person1 = """\
A is a 30-year-old woman living in San Francisco. She studied computer science at \
Stanford and worked for six years as a senior engineer at Google on \
large-scale distributed systems. She recently left to start an AI-powered healthcare \
analytics company and has already raised a $2M seed round from \
well-known Bay Area investors.
"""

person2 = """\
B is a 25-year-old man based in New York City. He graduated with a degree in \
marketing from NYU and has been working as a marketing manager at Apple for \
the past three years. He is trying to launch a social media app. Before apple, \
he was a product manager at Facebook.
"""

person3 = """\
C is stay in Los Angeles. He is a practicing medical doctor at UCLA \
and is working on a remote patient monitoring platform. He has limited \
technical knowledge and no startup experience, relying heavily on contractors \
for development. He is also a big fan of the Lakers.
"""

person4 = """\
D is a 40-year-old man living in Chicago. He studied law at the University of \
Chicago and has built a career as a corporate lawyer specializing in \
mergers and acquisitions. He is exploring a legal-tech startup idea but is \
still working full-time at his law firm and has no technical or entrepreneurial \
background.
"""

person5 = """\
E is a 28-year-old woman in San Francisco. She studied computer engineering at \
UC Berkeley and worked as a software engineer at a YC-backed fintech startup \
that scaled rapidly. She is now building her own fintech product for underbanked \
communities and has early traction with pilot customers in Latin America.
"""

person6 = """\
F is a 32-year-old man based in New York City. He earned his MBA from Columbia \
Business School after working in marketing roles at Apple and Spotify. He is \
now working on a consumer subscription box startup, but customer acquisition costs \
have been high, and he is struggling to attract investors without stronger traction.
"""

person7 = """\
G is a 27-year-old woman living in Austin, Texas. She studied industrial engineering \
at MIT and later worked as a product manager at Amazon, focusing on supply chain \
logistics. She has teamed up with two cofounders from her professional network to \
launch a logistics automation startup and recently joined a prominent accelerator.
"""

person8 = """\
H has worked in 7 companies, in 3 different industries. He is currently a product \
manager at a startup in the fintech industry. He is looking to launch a new \
product in the edutech industry.
"""

In [4]:
X = pd.DataFrame(
    {
        "data": [
            person1,
            person2,
            person3,
            person4,
            person5,
            person6,
            person7,
            person8,
        ]
    }
)
y = ["YES", "NO", "NO", "YES", "NO", "NO", "YES", "NO"]

# Policy Induction

In [5]:
config = WeightTrainerConfig(cv_folds=2)
pi = PolicyInduction(
        gen_llmc=[
            GoogleChoice(model="gemini-2.5-flash"),
            OpenAIChoice(model="gpt-4o-mini"),
        ],
        predict_llmc=[
            GoogleChoice(model="gemini-2.5-flash"),
            OpenAIChoice(model="gpt-4o-mini"),
        ],
        config=config
    )

In [6]:
instructions_tem = await pi.set_task(
        task_description="Predict if a startup founder will be successful "
        "or fail based on their background.",
    )
print(instructions_tem)

2025-10-18 02:46:37,405 INFO think_reason_learn.policy_induction._policy_induction: Generated policy generation instructions with confidence 0.9523611757070704
You are an expert policy refiner and extender for binary classification tasks. Your objective is to improve and generalize existing classification policies by incorporating insights from new data, rather than discarding the original policies.

You will be provided with:
1.  A **Task Description**: Outlining the goal of the classification.
2.  **Existing Policies**: A set of policies generated in previous rounds.
3.  **New Data Samples**: Recently collected data points with their corresponding binary labels.

Your task is to analyze the new data samples in the context of the task description and the existing policies. Use the new data to:
*   **Refine existing policies**: Clarify ambiguous logic, make them more precise, and improve their general applicability across different cases.
*   **Extend policies**: Identify new, broader 

In [7]:
pi = await pi.fit(X, y, reset=True)

2025-10-18 02:46:37,420 INFO think_reason_learn.policy_induction._policy_induction: Generating Policies
2025-10-18 02:46:58,477 INFO think_reason_learn.policy_induction._policy_induction: Generated policies: 8
2025-10-18 02:46:58,479 INFO think_reason_learn.policy_induction._policy_induction: Scoring Policies
2025-10-18 02:48:50,139 INFO think_reason_learn.policy_induction._policy_induction: Setting policy weight
2025-10-18 02:48:50,153 INFO think_reason_learn.policy_induction._policy_induction: Fitting weights: 8 samples × 8 models
2025-10-18 02:48:50,628 INFO think_reason_learn.policy_induction._policy_induction: C=0.001 mean F0.5=0.47009
2025-10-18 02:48:51,079 INFO think_reason_learn.policy_induction._policy_induction: C=0.01 mean F0.5=0.47009
2025-10-18 02:48:51,520 INFO think_reason_learn.policy_induction._policy_induction: C=0.1 mean F0.5=0.47009
2025-10-18 02:48:51,967 INFO think_reason_learn.policy_induction._policy_induction: C=1 mean F0.5=0.69444
2025-10-18 02:48:52,408 INFO

In [8]:
pi.get_memory()

,policy,predictions
0,"Founders possessing deep, directly relevant do...",0 YES 1 YES 2 YES 3 YES 4 YES 5...
1,Founders who have graduated from top-tier educ...,0 YES 1 YES 2 YES 3 YES 4 YES 5...
2,"Securing early external validation, such as si...",0 YES 1 NO 2 NO 3 NO 4 NO 5...
3,A significant gap in critical expertise requir...,0 YES 1 NO 2 NO 3 NO 4 YES 5...
4,Founders experiencing high customer acquisitio...,0 YES 1 YES 2 NO 3 NO 4 YES 5...
5,Founders with a very broad or frequently chang...,0 NO 1 YES 2 YES 3 YES 4 NO 5...
6,Ventures entering highly competitive markets w...,0 NO 1 NO 2 NO 3 NO 4 YES 5...
7,Even founders with strong technical skills and...,0 NO 1 NO 2 NO 3 NO 4 NO 5...


In [9]:
async for sample_index, results, final_answer, token_counter in pi.predict(X):
    print(f"Sample {sample_index}, Predict: {final_answer}")

Sample 0, Predict: YES
Sample 2, Predict: NO
Sample 1, Predict: NO
Sample 3, Predict: NO
Sample 4, Predict: NO
Sample 5, Predict: NO
Sample 7, Predict: NO
Sample 6, Predict: NO


# Saving

In [10]:
pi.save("example_policy_induction", for_production=True)

# Loading

In [11]:
loaded_pi = PolicyInduction.load("example_policy_induction")

In [12]:
async for idx, policy_vector, answer, token_counter in pi.predict(X):
    print(idx, answer)

0 YES
1 NO
3 NO
2 NO
5 NO
4 NO
6 NO
7 NO
